In [2]:
import cv2
import numpy as np

In [21]:
image_path = r'data\gray_image\sample2.png'

img = cv2.imread(image_path)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (1500,850))
img_gray = cv2.resize(img_gray, (1500,850))

cv2.imshow("img", img)
cv2.imshow("gray", img_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [74]:
# roi = img[0:780, 120:1010]
# roi_gray = img_gray[0:880, 120:1010]

# cv2.imshow("ROI", roi_gray)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

### Bilateral Filter

In [24]:
def nothing(x):
    pass

cv2.namedWindow("BilaterFilter")
cv2.createTrackbar("d", "BilaterFilter", 3, 50, nothing)
cv2.createTrackbar("sigmaColor", "BilaterFilter", 0, 500, nothing)
cv2.createTrackbar("sigmaSpace", "BilaterFilter", 0, 500, nothing)

while True:
    d = cv2.getTrackbarPos('d', 'BilaterFilter')
    sigmaColor = cv2.getTrackbarPos('sigmaColor', 'BilaterFilter')
    sigmaSpace = cv2.getTrackbarPos('sigmaSpace', 'BilaterFilter')

    
    bi_f = cv2.bilateralFilter(img_gray, d, sigmaColor, sigmaSpace)
    cv2.imshow("BilaterFilter", bi_f)
    k = cv2.waitKey(1)
    if k == ord("q"):
        break

cv2.destroyAllWindows()

### Threshold Detection

In [25]:
def nothing(x):
    pass

cv2.namedWindow("Threshold")
cv2.createTrackbar("a", "Threshold", 0, 255, nothing)
cv2.createTrackbar("b", "Threshold", 10, 255, nothing)

while True:
    a = cv2.getTrackbarPos('a', 'Threshold')
    b = cv2.getTrackbarPos('b', 'Threshold')
    
    
    ret,thresh = cv2.threshold(bi_f,a,b,cv2.THRESH_BINARY_INV)
    cv2.imshow("Threshold", thresh)
    k = cv2.waitKey(1)
    if k == ord("q"):
        break

cv2.destroyAllWindows()

In [26]:
cv2.imshow('Threshold', thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [29]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,1))
dilated = cv2.dilate(thresh, kernel, iterations=1)
eroded = cv2.erode(dilated, kernel, iterations=4)

cv2.imshow('Morphological', dilated)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [30]:
cnts,hier = cv2.findContours(dilated,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
# print("Number of contour==",cnts,"\ntotal contour==",len(cnts))
# print("Hierarchy==\n",hier)

In [40]:
# roi_gray_g = cv2.COLOR_GRAY2BGR(roi_gray)
img_c = cv2.drawContours(img,cnts,-1,(0,0,255),1)

cv2.imshow("Contour===",img_c)
# cv2.imwrite(r'data/gray_image/output_1.png', img_c)
# cv2.imshow("thresh==",thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Find the preferal of the contours

In [9]:
sum = 0
for c in cnts:
    perimeter = cv2.arcLength(c,False)
    sum = sum + perimeter

In [53]:
len_inch = sum/96 # 96 is the DPI of the image
len_feet = len_inch*8 # Scale 1 inch is equal to 8 feet 

# 5% error 
per_error = len_feet*0.05

print(f'Pipe length in Inches: {np.round(len_inch,2)}\n')
print(f'approx Pipe length in Feet: {np.round(len_feet, 2)}\n')
print(f"Pipe Length in feet with 5% error probablity\nLower range:{np.round(len_feet-per_error,2)}\nUpper range:{np.round(len_feet+per_error,2)}")

Pipe length in Inches: 33.3

approx Pipe length in Feet: 266.42

Pipe Length in feet with 5% error probablity
Lower range:253.1
Upper range:279.74


In [45]:
len_feet*0.05

13.320833333333335